In [1]:
import sys; sys.path.insert(0, '../../invert')
from invert.forward import get_info, create_forward_model
import mne
import pickle as pkl
import os
import numpy as np
import matplotlib.pyplot as plt
os.makedirs("forward_models", exist_ok=True)
verbose = 0

In [4]:
samplings = {
    "fine": "oct6", 
    "coarse": "ico4"}

# Clean Forward Models

In [7]:
n_chans = 256
info = get_info(f"biosemi{n_chans}")

fn = f"info_biosemi{n_chans}.pkl"
with open(fn, 'wb') as f:
    pkl.dump(info, f)

# get fsaverage forward model
ratio = 80
conductivity = [0.33, 0.33/ratio, 0.33]

fs_dir = mne.datasets.fetch_fsaverage(verbose=verbose)
subject = 'fsaverage'
subjects_dir = os.path.dirname(fs_dir)
surfaces = mne.make_bem_model(subject, ico=4, conductivity=conductivity, verbose=verbose)
bem = mne.make_bem_solution(surfaces, verbose=verbose)

trans = os.path.join(fs_dir, 'bem', 'fsaverage-trans.fif')
fwds = dict()
srcs = dict()
for key, sampling in samplings.items():
    
    # Create and save Source Model
    src = mne.setup_source_space(subject, spacing=sampling, surface='white',
                                        subjects_dir=subjects_dir, add_dist=False,
                                        n_jobs=1, verbose=verbose)
    # Forward Model
    fwd = mne.make_forward_solution(info, trans=trans, src=src,
                                    bem=bem, eeg=True, mindist=5.0, n_jobs=1,
                                    verbose=verbose)
    
    fwds[f"""Clean_{key}"""] = fwd
    srcs[key] = src
    fn = f"forward_models/fsaverage_biosemi{n_chans}_Clean_{key}-fwd.fif"
    mne.write_forward_solution(fn, fwd, overwrite=True)
    print(f"Saved {fn}")


    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
Saved forward_models/fsaverage_biosemi256_Clean_fine-fwd.fif
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
Saved forward_models/fsaverage_biosemi256_Clean_coarse-fwd.fif


# Forward models with conductivity error

In [8]:
key = "fine"
for ratio in (50, 30):
    conductivity = [0.33, 0.33/ratio, 0.33]
    
    surfaces = mne.make_bem_model(subject, ico=4, conductivity=conductivity, subjects_dir=subjects_dir, verbose=verbose)
    bem = mne.make_bem_solution(surfaces, verbose=verbose)
    # Forward Model
    fwd = mne.make_forward_solution(info, trans=trans, src=srcs[key],
                                    bem=bem, eeg=True, mindist=5.0, n_jobs=1,
                                    verbose=verbose)
    
    fwds[f"""Ratio-{ratio}_{key}"""] = fwd

    fn = f"forward_models/fsaverage_biosemi{n_chans}_Ratio-{ratio}_{key}-fwd.fif"
    mne.write_forward_solution(fn, fwd, overwrite=True)
    print(f"Saved {fn}")


    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
Saved forward_models/fsaverage_biosemi256_Ratio-50_fine-fwd.fif
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
Saved forward_models/fsaverage_biosemi256_Ratio-30_fine-fwd.fif
